# 04 Robustness Checks

This notebook explores robustness checks for the Double ML results, including alternative machine learning models and subsampling by region or income group.


In [2]:
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages (module name, pip name if different)
required_packages = [
    ("statsmodels", "statsmodels"),
    ("xgboost", "xgboost"),
    ("sklearn", "scikit-learn"),
    ("scipy", "scipy"),
    ("pandas", "pandas"),
    ("numpy", "numpy"),
]

for module_name, pip_name in required_packages:
    install_if_missing(module_name)


  Using cached statsmodels-0.14.5-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.5 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached statsmodels-0.14.5-cp39-cp39-macosx_11_0_arm64.whl (9.8 MB)
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]


In [4]:
"""
This script performs a comprehensive causal machine‑learning analysis of the
relationship between lagged climate vulnerability and sovereign bond spreads.
It includes:

1.  Data preparation (loading, renaming columns, computing lags and dummy
    variables).
2.  Baseline Double Machine Learning (DML) with leave‑one‑year‑out cross‑fitting
    for a binary “high‑spread” event (top 10 % of spreads).
3.  Robustness checks: random permutation placebo, heterogeneity by income and
    governance quality, regional splits.
4.  First‑difference DML to remove time‑invariant fixed effects.
5.  Instrumental‑variable (2SLS) regression using the second lag of
    vulnerability as an instrument.
6.  Linear quantile regressions at the 90th, 95th and 99th percentiles.
7.  Quantile gradient boosting regressions at the 95th and 99th percentiles and
    estimation of the marginal effect of vulnerability on these conditional
    quantiles.

The code is designed to be run in a notebook or standalone Python script.  It
prints out key results for inclusion in an academic paper.
"""

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import norm, t


def load_and_prepare_data(csv_path: str) -> pd.DataFrame:
    """Load the dataset, rename columns to avoid spaces, compute lags and dummy vars.

    Args:
        csv_path: Path to the CSV file containing the baseline panel.

    Returns:
        A processed DataFrame ready for analysis.
    """
    df = pd.read_csv(csv_path).sort_values(["iso3c", "year"]).reset_index(drop=True)
    # Replace spaces in column names with underscores
    df = df.rename(columns=lambda x: x.strip().replace(" ", "_"))
    # Compute lagged vulnerability (lag 1 and lag 2)
    df["vulnerability_lag1"] = df.groupby("iso3c")["vulnerability"].shift(1)
    df["vulnerability_lag2"] = df.groupby("iso3c")["vulnerability"].shift(2)
    # Create high_spread event: top 10 % of spreads
    thr = df["sovereign_spread"].quantile(0.90)
    df["high_spread"] = (df["sovereign_spread"] >= thr).astype(int)
    # Create region dummy variables
    region_dummies = pd.get_dummies(df["region"], prefix="reg")
    df = pd.concat([df, region_dummies], axis=1)
    # Compute an overall WGI score per row and classify governance groups
    wgi_cols = [c for c in df.columns if c.startswith("wgi")]
    df["wgi_score"] = df[wgi_cols].mean(axis=1)
    avg_wgi_country = df.groupby("iso3c")["wgi_score"].mean()
    median_wgi = avg_wgi_country.median()
    df["governance_group"] = df["iso3c"].apply(
        lambda iso: "High_Gov" if avg_wgi_country[iso] > median_wgi else "Low_Gov"
    )
    # Compute an income classification based on median GDP per capita across countries
    avg_gdp_country = df.groupby("iso3c")["gdp_per_capita"].mean()
    median_gdp = avg_gdp_country.median()
    df["income_group"] = df["iso3c"].apply(
        lambda iso: "High" if avg_gdp_country[iso] > median_gdp else "Low"
    )
    return df


def run_loyo_dml(Y, T, X, years, groups, n_neighbors=5, n_estimators=100, random_state=7):
    """Run leave‑one‑year‑out DML for a binary outcome.

    Args:
        Y: array of outcome values (0/1 for tail risk event).
        T: array of treatment values (lagged vulnerability).
        X: 2D array of covariates.
        years: array of year identifiers for cross‑fitting splits.
        groups: array of cluster identifiers (countries).
        n_neighbors: number of neighbors for KNN imputation.
        n_estimators: number of trees in XGBoost models.
        random_state: random seed for reproducibility.

    Returns:
        theta, se, pval – the treatment effect, standard error and p‑value.
    """
    N = len(Y)
    Yres = np.zeros(N)
    Tres = np.zeros(N)
    unique_years = np.unique(years)
    for yr in unique_years:
        test_idx = np.where(years == yr)[0]
        train_idx = np.where(years != yr)[0]
        # Impute and standardize covariates within fold
        imputer = KNNImputer(n_neighbors=n_neighbors)
        scaler = StandardScaler()
        X_train = imputer.fit_transform(X[train_idx])
        X_test = imputer.transform(X[test_idx])
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        T_train = T[train_idx]
        Y_train = Y[train_idx]
        # Treatment model (regressor)
        mdl_T = xgb.XGBRegressor(
            n_estimators=n_estimators,
            max_depth=4,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=random_state,
        )
        mdl_T.fit(X_train, T_train)
        p_hat = mdl_T.predict(X_test)
        # Outcome model (classifier)
        mdl_Y = xgb.XGBClassifier(
            n_estimators=n_estimators,
            max_depth=4,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=random_state,
        )
        mdl_Y.fit(X_train, Y_train)
        m_hat = mdl_Y.predict_proba(X_test)[:, 1]
        # Residuals
        Yres[test_idx] = Y[test_idx] - m_hat
        Tres[test_idx] = T[test_idx] - p_hat
    # Second stage regression with cluster‑robust SEs
    res = sm.OLS(Yres, Tres).fit(cov_type="cluster", cov_kwds={"groups": groups})
    theta = res.params[0]
    se = res.bse[0]
    pval = res.pvalues[0]
    return theta, se, pval


def dml_tail_risk_analysis(df: pd.DataFrame, q=0.10):
    """Run baseline tail‑risk DML and heterogeneity checks.

    Args:
        df: processed DataFrame.
        q: quantile to define the high‑spread event (default 0.10 for top 10 %).
    Returns:
        A dictionary of results.
    """
    # Identify high‑spread event based on quantile q
    thr = df["sovereign_spread"].quantile(1 - q)
    df["high_spread"] = (df["sovereign_spread"] >= thr).astype(int)
    # Define feature set
    wgi_cols = [c for c in df.columns if c.startswith("wgi")]
    macro_cols = [
        "cpi_yoy",
        "gdp_annual_growth_rate",
        "gdp_per_capita",
        "gross_gdp",
        "debt_to_gdp",
        "deficit_to_gdp",
        "current_account_balance",
        "population",
        "mineral_rent",
        "gain",
    ]
    cat_cols = [c for c in df.columns if c.startswith("reg_")]
    X_cols = wgi_cols + macro_cols + cat_cols
    # Baseline mask
    mask_all = df["vulnerability_lag1"].notna() & df["high_spread"].notna()
    Y = df.loc[mask_all, "high_spread"].values
    T = df.loc[mask_all, "vulnerability_lag1"].values
    years = df.loc[mask_all, "year"].values
    groups = df.loc[mask_all, "iso3c"].values
    X = df.loc[mask_all, X_cols].values
    # Baseline DML
    theta, se, pval = run_loyo_dml(Y, T, X, years, groups, n_neighbors=5, n_estimators=200)
    results = {"baseline_theta": theta, "baseline_se": se, "baseline_pval": pval}
    # Placebo: random permutation of treatment
    np.random.seed(42)
    T_perm = np.random.permutation(T)
    theta_perm, se_perm, pval_perm = run_loyo_dml(Y, T_perm, X, years, groups, n_neighbors=5, n_estimators=200)
    results.update({
        "perm_theta": theta_perm,
        "perm_se": se_perm,
        "perm_pval": pval_perm,
    })
    # Heterogeneity by income
    for group_name in ["Low", "High"]:
        mask = (df["income_group"] == group_name) & mask_all
        Yg, Tg, Xg = df.loc[mask, "high_spread"].values, df.loc[mask, "vulnerability_lag1"].values, df.loc[mask, X_cols].values
        yearsg = df.loc[mask, "year"].values
        groupsg = df.loc[mask, "iso3c"].values
        theta_g, se_g, p_g = run_loyo_dml(Yg, Tg, Xg, yearsg, groupsg, n_neighbors=5, n_estimators=200)
        results[f"income_{group_name.lower()}_theta"] = theta_g
        results[f"income_{group_name.lower()}_se"] = se_g
        results[f"income_{group_name.lower()}_pval"] = p_g
    # Income difference significance
    theta_low = results["income_low_theta"]
    se_low = results["income_low_se"]
    theta_high = results["income_high_theta"]
    se_high = results["income_high_se"]
    diff = theta_low - theta_high
    se_diff = np.sqrt(se_low ** 2 + se_high ** 2)
    z_score = diff / se_diff
    p_diff = 2 * (1 - norm.cdf(abs(z_score)))
    results.update({
        "income_diff": diff,
        "income_diff_se": se_diff,
        "income_diff_pval": p_diff,
    })
    # Heterogeneity by governance quality
    for group_name in ["Low_Gov", "High_Gov"]:
        mask = (df["governance_group"] == group_name) & mask_all
        Yg, Tg, Xg = df.loc[mask, "high_spread"].values, df.loc[mask, "vulnerability_lag1"].values, df.loc[mask, X_cols].values
        yearsg = df.loc[mask, "year"].values
        groupsg = df.loc[mask, "iso3c"].values
        theta_g, se_g, p_g = run_loyo_dml(Yg, Tg, Xg, yearsg, groupsg, n_neighbors=5, n_estimators=200)
        results[f"gov_{group_name.lower()}_theta"] = theta_g
        results[f"gov_{group_name.lower()}_se"] = se_g
        results[f"gov_{group_name.lower()}_pval"] = p_g
    # Governance difference significance
    theta_low_g = results["gov_low_gov_theta"]
    se_low_g = results["gov_low_gov_se"]
    theta_high_g = results["gov_high_gov_theta"]
    se_high_g = results["gov_high_gov_se"]
    diff_g = theta_low_g - theta_high_g
    se_diff_g = np.sqrt(se_low_g ** 2 + se_high_g ** 2)
    z_score_g = diff_g / se_diff_g
    p_diff_g = 2 * (1 - norm.cdf(abs(z_score_g)))
    results.update({
        "gov_diff": diff_g,
        "gov_diff_se": se_diff_g,
        "gov_diff_pval": p_diff_g,
    })
    # Regional splits (only regions with >100 observations)
    regions = df["region"].unique()
    region_results = {}
    for reg in regions:
        mask_reg = (df["region"] == reg) & mask_all
        n_reg = mask_reg.sum()
        if n_reg > 100:
            Yg, Tg, Xg = df.loc[mask_reg, "high_spread"].values, df.loc[mask_reg, "vulnerability_lag1"].values, df.loc[mask_reg, X_cols].values
            yearsg = df.loc[mask_reg, "year"].values
            groupsg = df.loc[mask_reg, "iso3c"].values
            theta_g, se_g, p_g = run_loyo_dml(Yg, Tg, Xg, yearsg, groupsg, n_neighbors=5, n_estimators=200)
            region_results[reg] = (theta_g, se_g, p_g, n_reg)
    results["region_results"] = region_results
    return results


def first_difference_dml(df: pd.DataFrame) -> tuple:
    """Run first‑difference DML for the change in high_spread vs change in vulnerability.

    This removes time‑invariant country fixed effects following Clarke & Polselli.
    """
    # Compute differenced variables per country
    # List of covariates to difference
    feature_cols = [
        c
        for c in df.columns
        if c.startswith("wgi")
        or c
        in [
            "cpi_yoy",
            "gdp_annual_growth_rate",
            "gdp_per_capita",
            "gross_gdp",
            "debt_to_gdp",
            "deficit_to_gdp",
            "current_account_balance",
            "population",
            "mineral_rent",
            "gain",
        ]
    ]
    diff_dfs = []
    for iso, g in df.groupby("iso3c"):
        g = g.sort_values("year")
        # Compute first differences of treatment, outcome and controls
        diff_data = {f"diff_{col}": g[col].diff() for col in feature_cols}
        diff_data["diffY"] = g["high_spread"].diff()
        diff_data["diffT"] = g["vulnerability_lag1"].diff()
        diff_data["iso3c"] = iso
        diff_data["year"] = g["year"].values
        diff_df = pd.DataFrame(diff_data)
        diff_dfs.append(diff_df)
    fd_df = pd.concat(diff_dfs).dropna()
    # Prepare arrays
    Y = fd_df["diffY"].values
    T = fd_df["diffT"].values
    years = fd_df["year"].values
    groups = fd_df["iso3c"].values
    X = fd_df[[c for c in fd_df.columns if c.startswith("diff_") and c not in ["diffY", "diffT"]]].values
    # Run LOYO DML on differenced data using regressor for both Y and T
    N = len(Y)
    Yres = np.zeros(N)
    Tres = np.zeros(N)
    for yr in np.unique(years):
        test_idx = np.where(years == yr)[0]
        train_idx = np.where(years != yr)[0]
        imputer = KNNImputer(n_neighbors=5)
        scaler = StandardScaler()
        X_train = imputer.fit_transform(X[train_idx])
        X_test = imputer.transform(X[test_idx])
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        T_train = T[train_idx]
        Y_train = Y[train_idx]
        mdl_T = xgb.XGBRegressor(
            n_estimators=100,
            max_depth=4,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=7,
        )
        mdl_T.fit(X_train, T_train)
        p_hat = mdl_T.predict(X_test)
        mdl_Y = xgb.XGBRegressor(
            n_estimators=100,
            max_depth=4,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=7,
        )
        mdl_Y.fit(X_train, Y_train)
        m_hat = mdl_Y.predict(X_test)
        Yres[test_idx] = Y[test_idx] - m_hat
        Tres[test_idx] = T[test_idx] - p_hat
    res = sm.OLS(Yres, Tres).fit(cov_type="cluster", cov_kwds={"groups": groups})
    return res.params[0], res.bse[0], res.pvalues[0]


def instrumental_variable_analysis(df: pd.DataFrame) -> tuple:
    """Run a two‑stage least squares (2SLS) regression for the high_spread indicator.

    The second lag of vulnerability is used as an instrument for the first lag.
    """
    # Select rows with all needed variables non‑null
    wgi_cols = [c for c in df.columns if c.startswith("wgi")]
    macro_cols = [
        "cpi_yoy",
        "gdp_annual_growth_rate",
        "gdp_per_capita",
        "gross_gdp",
        "debt_to_gdp",
        "deficit_to_gdp",
        "current_account_balance",
        "population",
        "mineral_rent",
        "gain",
    ]
    cat_cols = [c for c in df.columns if c.startswith("reg_")]
    X_cols = wgi_cols + macro_cols + cat_cols
    mask = df["high_spread"].notna() & df["vulnerability_lag1"].notna() & df["vulnerability_lag2"].notna()
    sub = df.loc[mask].reset_index(drop=True)
    # Impute missing covariates with KNN
    imputer = KNNImputer(n_neighbors=5)
    X_imp = pd.DataFrame(imputer.fit_transform(sub[X_cols]), columns=X_cols)
    # Stage 1: regress vulnerability_lag1 on vulnerability_lag2 and controls
    X1 = sm.add_constant(pd.concat([sub["vulnerability_lag2"], X_imp], axis=1))
    mod1 = sm.OLS(sub["vulnerability_lag1"], X1).fit()
    T_hat = mod1.fittedvalues.rename("T_hat")
    # Stage 2: regress high_spread on T_hat and controls
    X2 = sm.add_constant(pd.concat([T_hat, X_imp], axis=1))
    mod2 = sm.OLS(sub["high_spread"], X2).fit(cov_type="cluster", cov_kwds={"groups": sub["iso3c"]})
    coef = mod2.params["T_hat"]
    se = mod2.bse["T_hat"]
    pval = mod2.pvalues["T_hat"]
    return coef, se, pval


def quantile_regressions(df: pd.DataFrame):
    """Run linear quantile regressions on sovereign spreads.
    Returns a dict with coefficients and p‑values for vulnerability at q=0.90,0.95,0.99.
    """
    import statsmodels.formula.api as smf
    df_qr = df.copy()
    # Remove rows with any missing in outcome, treatment or controls
    wgi_cols = [c for c in df_qr.columns if c.startswith("wgi")]
    macro_cols = [
        "cpi_yoy",
        "gdp_annual_growth_rate",
        "gdp_per_capita",
        "gross_gdp",
        "debt_to_gdp",
        "deficit_to_gdp",
        "current_account_balance",
        "population",
        "mineral_rent",
        "gain",
    ]
    model_formula = "sovereign_spread ~ vulnerability_lag1 + " + " + ".join(wgi_cols + macro_cols)
    # Drop rows with NaNs in formula variables
    mask = df_qr[["sovereign_spread", "vulnerability_lag1"] + wgi_cols + macro_cols].notnull().all(axis=1)
    df_clean = df_qr.loc[mask]
    results = {}
    for q in [0.90, 0.95, 0.99]:
        try:
            mod = smf.quantreg(model_formula, df_clean).fit(q=q)
            coef = mod.params["vulnerability_lag1"]
            pval = mod.pvalues["vulnerability_lag1"]
            results[q] = (coef, pval)
        except Exception:
            results[q] = (np.nan, np.nan)
    return results


def quantile_gradient_boosting(df: pd.DataFrame):
    """Estimate the effect of vulnerability on the 95th and 99th percentiles of spreads
    using quantile gradient boosting.  Returns a dict with the estimated mean
    change in predicted quantiles (per 1‑SD increase in vulnerability) and its
    t‑statistic/p‑value.
    """
    # Prepare features
    wgi_cols = [c for c in df.columns if c.startswith("wgi")]
    macro_cols = [
        "cpi_yoy",
        "gdp_annual_growth_rate",
        "gdp_per_capita",
        "gross_gdp",
        "debt_to_gdp",
        "deficit_to_gdp",
        "current_account_balance",
        "population",
        "mineral_rent",
        "gain",
    ]
    cat_cols = [c for c in df.columns if c.startswith("reg_")]
    X_cols = ["vulnerability_lag1"] + wgi_cols + macro_cols + cat_cols
    mask = df[X_cols + ["sovereign_spread"]].notnull().all(axis=1)
    sub = df.loc[mask].reset_index(drop=True)
    imputer = KNNImputer(n_neighbors=5)
    X = pd.DataFrame(imputer.fit_transform(sub[X_cols]), columns=X_cols)
    Y = sub["sovereign_spread"].values
    # Compute standard deviation of vulnerability for scaling
    std_vul = X["vulnerability_lag1"].std()
    results = {}
    for alpha in [0.95, 0.99]:
        gbr = GradientBoostingRegressor(
            loss="quantile",
            alpha=alpha,
            n_estimators=300,
            max_depth=3,
            learning_rate=0.05,
            random_state=42,
        )
        gbr.fit(X, Y)
        pred_base = gbr.predict(X)
        X_high = X.copy()
        X_high["vulnerability_lag1"] = X_high["vulnerability_lag1"] + std_vul
        pred_high = gbr.predict(X_high)
        diff = pred_high - pred_base
        mean_diff = diff.mean()
        # Standard error of the mean difference
        se_diff = diff.std(ddof=1) / np.sqrt(len(diff))
        t_stat = mean_diff / se_diff
        pval = 2 * (1 - t.cdf(np.abs(t_stat), df=len(diff) - 1))
        results[alpha] = {
            "mean_diff": mean_diff,
            "se_diff": se_diff,
            "t_stat": t_stat,
            "pval": pval,
        }
    return results


In [ ]:


def main():
    # Path to CSV (adjust as necessary)
    csv_path = "baseline_with_gain_population_mineral_regions.csv"
    df = load_and_prepare_data(csv_path)
    # 1. Baseline DML and heterogeneity
    dml_results = dml_tail_risk_analysis(df, q=0.10)
    print("\n=== Baseline DML and Heterogeneity Results ===")
    print(
        f"Baseline effect: θ = {dml_results['baseline_theta']:.4f}, SE = {dml_results['baseline_se']:.4f}, p = {dml_results['baseline_pval']:.4f}"
    )
    print(
        f"Placebo (permuted treatment): θ = {dml_results['perm_theta']:.4f}, SE = {dml_results['perm_se']:.4f}, p = {dml_results['perm_pval']:.4f}"
    )
    print("\nIncome heterogeneity:")
    print(
        f"  Low income: θ = {dml_results['income_low_theta']:.4f}, SE = {dml_results['income_low_se']:.4f}, p = {dml_results['income_low_pval']:.4f}"
    )
    print(
        f"  High income: θ = {dml_results['income_high_theta']:.4f}, SE = {dml_results['income_high_se']:.4f}, p = {dml_results['income_high_pval']:.4f}"
    )
    print(
        f"  Difference (low – high): diff = {dml_results['income_diff']:.4f}, SE = {dml_results['income_diff_se']:.4f}, p = {dml_results['income_diff_pval']:.4f}"
    )
    print("\nGovernance heterogeneity:")
    print(
        f"  Low governance: θ = {dml_results['gov_low_gov_theta']:.4f}, SE = {dml_results['gov_low_gov_se']:.4f}, p = {dml_results['gov_low_gov_pval']:.4f}"
    )
    print(
        f"  High governance: θ = {dml_results['gov_high_gov_theta']:.4f}, SE = {dml_results['gov_high_gov_se']:.4f}, p = {dml_results['gov_high_gov_pval']:.4f}"
    )
    print(
        f"  Difference (low – high): diff = {dml_results['gov_diff']:.4f}, SE = {dml_results['gov_diff_se']:.4f}, p = {dml_results['gov_diff_pval']:.4f}"
    )
    print("\nRegion heterogeneity (regions with >100 observations):")
    for reg, (theta, se, p, n_reg) in dml_results["region_results"].items():
        print(f"  {reg}: θ = {theta:.3f}, SE = {se:.3f}, p = {p:.3f}, n = {n_reg}")
    # 2. First‑difference DML
    fd_theta, fd_se, fd_p = first_difference_dml(df)
    print("\n=== First‑difference DML ===")
    print(f"θ = {fd_theta:.4f}, SE = {fd_se:.4f}, p = {fd_p:.4f}")
    # 3. Instrumental variables analysis
    iv_coef, iv_se, iv_p = instrumental_variable_analysis(df)
    print("\n=== Instrumental‑Variable (2SLS) Analysis ===")
    print(f"Coef = {iv_coef:.4f}, SE = {iv_se:.4f}, p = {iv_p:.4f}")
    # 4. Linear quantile regression results
    qr_results = quantile_regressions(df)
    print("\n=== Linear Quantile Regression ===")
    for q, (coef, pval) in qr_results.items():
        print(f"  Quantile {int(q*100)}%: coef = {coef:.6f}, p = {pval:.4e}")
    # 5. Quantile gradient boosting results
    qgb_results = quantile_gradient_boosting(df)
    print("\n=== Quantile Gradient Boosting (95th and 99th percentiles) ===")
    for alpha, stats in qgb_results.items():
        print(
            f"  α = {alpha:.2f}: mean ΔQ = {stats['mean_diff']:.4f}, SE = {stats['se_diff']:.4f}, t = {stats['t_stat']:.2f}, p = {stats['pval']:.4f}"
        )


if __name__ == "__main__":
    main()
